In [ ]:
# %% Deep learning - Section 10.95
#    Loss functions in PyTorch

# This code pertains a deep learning course provided by Mike X. Cohen on Udemy:
#   > https://www.udemy.com/course/deeplearning_x
# The "base" code in this repository is adapted (with very minor modifications)
# from code developed by the course instructor (Mike X. Cohen), while the
# "exercises" and the "code challenges" contain more original solutions and
# creative input from my side. If you are interested in DL (and if you are
# reading this statement, chances are that you are), go check out the course, it
# is singularly good.


In [ ]:
# %% Libraries and modules
import numpy               as np
import matplotlib.pyplot   as plt
import torch
import torch.nn            as nn
import seaborn             as sns
import copy
import torch.nn.functional as F
import pandas              as pd
import scipy.stats         as stats
import time

from torch.utils.data                 import DataLoader,TensorDataset
from sklearn.model_selection          import train_test_split
from google.colab                     import files
from torchsummary                     import summary
from IPython                          import display
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')


In [ ]:
# %% Mean-squared error (MSE)

# Loss
loss_fun = nn.MSELoss()

yHat = torch.linspace(-2,2,101)
y    = torch.tensor(.5)

L = np.zeros(len(yHat))
for i,yH in enumerate(yHat):
    L[i] = loss_fun(yH,y)


# Plotting
phi = ( 1 + np.sqrt(5) ) / 2
fig = plt.figure(figsize=(6*phi,6))

plt.plot(yHat,L,linewidth=2,label='Loss')
plt.plot([y,y],[0,np.max(L)],'r--')

plt.legend()
plt.grid()
plt.xlabel('Predicted value')
plt.ylabel('Loss')
plt.title('MSE loss function')

plt.savefig('figure56_loss_functions_pytorch.png')

plt.show()

files.download('figure56_loss_functions_pytorch.png')


In [ ]:
# %% Binary cross-entropy (BCE)

# Loss
loss_fun = nn.BCELoss()

yHat = torch.linspace(0.001,0.999,101)
y1   = torch.tensor(0.)
y2   = torch.tensor(1.)

L = np.zeros((len(yHat),2))
for i,yH in enumerate(yHat):
    L[i,0] = loss_fun(yH,y1)
    L[i,1] = loss_fun(yH,y2)

# Plotting
phi = ( 1 + np.sqrt(5) ) / 2
fig = plt.figure(figsize=(6*phi,6))

plt.plot(yHat,L,linewidth=2)
plt.plot([y1,y1],[0,np.max(L)],'r--')
plt.plot([y2,y2],[0,np.max(L)],'r--')

plt.legend(['Correct = 0','Correct = 1'])
plt.grid()
plt.xlabel('Predicted value')
plt.ylabel('Loss')
plt.title('MSE loss function')
plt.yscale('log')

plt.savefig('figure57_loss_functions_pytorch.png')

plt.show()

files.download('figure57_loss_functions_pytorch.png')


In [ ]:
# %% BCE without probability transform (e.g. sigmoid if binary)

# Use a 'raw' output
yHat = torch.tensor(2.)
print(loss_fun(yHat,y2))

# Convert to probability with sigmoid
yHat = torch.sigmoid(yHat)
print(loss_fun(yHat,y2))


In [ ]:
# %% Nota bene

# However, PyTorch recommends using a single function that incorporates
# sigmoid+BCE due to increased numerical stability (in toy examples, numerical
# accuracy usually isn't a problem).
# See : https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html?highlight=nn%20bcewithlogitsloss#torch.nn.BCEWithLogitsLoss

# Thus, the recommended way to do it:
loss_fun = nn.BCEWithLogitsLoss()
yHat     = torch.tensor(2.)
print(loss_fun(yHat,y2))


In [ ]:
# %% Categorical cross-entropy (CCE)

# Loss
loss_fun = nn.CrossEntropyLoss()

yHat = torch.tensor([[1.,4,3]]) # output layer (3 units) before softmax

for i in range(3):
    correct_answer = torch.tensor([i])
    current_loss   = loss_fun(yHat,correct_answer).item()
    print(f'Loss for correct answer index {i} equals {current_loss:.4f}')


In [ ]:
# %% Repeat but with softmax
# Not correct because nn.CrossEntropyLoss() already applies softmax

sm      = nn.Softmax(dim=1)
yHat_sm = sm(yHat)

for i in range(3):
    correct_answer = torch.tensor([i])
    current_loss   = loss_fun(yHat_sm,correct_answer).item()
    print(f'Loss for correct answer index {i} equals {current_loss:.4f}')


In [ ]:
# %% Compare raw, softmax, and log-softmax outputs

log_sm     = nn.LogSoftmax(dim=1)
yHat_logsm = log_sm(yHat)

print(yHat)
print(yHat_sm)
print(yHat_logsm)


In [ ]:
# %% Create your own custom loss function
#    Not recommended unless you really need it

class my_loss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self,x,y):
        loss = torch.abs(x-y)
        return loss

# Test
custom_fun = my_loss()
custom_fun(torch.tensor(4),torch.tensor(5.2))
